# Flask を使った簡易Webアプリ

まずは必要なライブラリをインストール

In [ ]:
!pip install -r requirements.txt

## その１　とりあえず音声ファイルをアップロードできるようにする．

In [34]:
%%writefile templates/index.html

<!DOCTYPE html>
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Voice Chat App</title>
  </head>
  <body>
    <h1>Voice Chat App</h1>
    <button id="start">開始</button>
    <button id="stop" disabled>停止</button>
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
      document.addEventListener("DOMContentLoaded", () => {
        const startButton = document.getElementById("start");
        const stopButton = document.getElementById("stop");
        const transcriptionElement = document.getElementById("transcription");
        const aiResponseElement = document.getElementById("aiResponse");
        let mediaRecorder;
        let audioChunks = [];

        startButton.addEventListener("click", async () => {
          const stream = await navigator.mediaDevices.getUserMedia({
            audio: true,
          });
          mediaRecorder = new MediaRecorder(stream);

          mediaRecorder.ondataavailable = (event) => {
            audioChunks.push(event.data);
          };

          mediaRecorder.onstop = async () => {
            const audioBlob = new Blob(audioChunks, { type: "audio/webm" });
            audioChunks = [];

            const formData = new FormData();
            formData.append("audio", audioBlob, "recording.webm");

            fetch("/upload", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                transcriptionElement.textContent =
                  data.text || "認識できませんでした。";
                aiResponseElement.textContent =
                  data.ai_response || "AIの応答なし。";
              })
              .catch((error) => {
                console.error("Upload failed:", error);
                transcriptionElement.textContent = "エラーが発生しました。";
                aiResponseElement.textContent = "";
              });
          };

          mediaRecorder.start();
          startButton.disabled = true;
          stopButton.disabled = false;
        });

        stopButton.addEventListener("click", () => {
          mediaRecorder.stop();
          startButton.disabled = false;
          stopButton.disabled = true;
        });
      });
    </script>
  </body>
</html>


Overwriting templates/index.html


In [ ]:
%%writefile app01.py

from flask import Flask, request, jsonify, render_template
import os

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")  # フロントエンドのHTMLを表示

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app01.py


In [35]:
!python app01.py

 * Serving Flask app 'app01'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
127.0.0.1 - - [27/Feb/2025 00:10:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2025 00:10:12] "POST /upload HTTP/1.1" 200 -
^C


## その２　アップロードした音声ファイルをWavファイルに変換

In [ ]:
%%writefile app02.py

from flask import Flask, request, jsonify, render_template
import os
import subprocess

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")  # フロントエンドのHTMLを表示

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    convert_webm_to_wav(audio_path, "uploads/output.wav")
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})


def convert_webm_to_wav(input_path, output_path):
    command = [
        "ffmpeg",
        "-i", input_path,  # 入力ファイル
        "-ar", "16000",  # サンプリングレート 16kHz
        "-ac", "1",  # モノラル変換
        "-preset", "ultrafast",  # 速度最優先
        output_path
    ]
    subprocess.run(command, check=True)

# 使い方

if __name__ == "__main__":
    app.run(debug=True)


Writing app02.py


In [11]:
!python app02.py

 * Serving Flask app 'app02'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
127.0.0.1 - - [26/Feb/2025 22:49:28] "GET / HTTP/1.1" 200 -
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 

変換に少々時間取られるな．．．

## その３　アップロードを直接Wavファイルにする．

変換に少々時間がかかるのが気になるので，アップロードの段階で直接Wavファイルをアップできないか探ってみたら，Record.jsなるものがあるようだ．
https://github.com/mattdiamond/Recorderjs
Recorder.jsをダウンロードして
これをhtmlに組み込んでみる．

In [ ]:
%%writefile templates/index.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>
    <!-- <script src="recorder.js"></script> --> 
    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;
                

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("audio", audioBlob, "recorded_audio.wav");

                fetch("/uploads", {
                    method: "POST",
                    body: formData,
                    mode: "cors"  // CORSを許可
                })
                .then((response) => response.json())
                .then((data) => {
                    transcriptionElement.textContent =
                    data.text || "認識できませんでした。";
                    aiResponseElement.textContent =
                    data.ai_response || "AIの応答なし。";
                })
                .catch((error) => {
                    console.error("Upload failed:", error);
                    transcriptionElement.textContent = "エラーが発生しました。";
                    aiResponseElement.textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting templates/index.html


In [40]:
!python app01.py

6994.00s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


 * Serving Flask app 'app01'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
127.0.0.1 - - [27/Feb/2025 00:21:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2025 00:21:48] "GET / HTTP/1.1" 200 -
^C


CORSの問題でアップロードで弾かれているようだ．．．